In [4]:
from ipywidgets import FloatSlider, Layout
import ipywidgets as widgets
from PIL import Image
import numpy as np

from lib.cnn import ImageUtil, CNN, CNNTemplate, CNNSimulator
from lib.cnn_template import CNNTemplateLib
import lib.simulationui as gui

import numpy as np
import math
from lib.ode.algorithm import rk4_system_algorithm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import pandas as pd

from lib.ode import rk4_system_method
from lib.ode import rkacem_system_method
from lib.ode import rkhm_system_method
from lib.ode import rkhem_system_method
from lib.ode import rkcom_system_method
from lib.ode import rk55_system_method

In [10]:
def generate_empty_simulation_result_csv():
    simulation_result_dataset = {
        "simulation_name": [],
        "method_name": [],
        "step_size": [],
        "tolerance": [],
        "total_iteration": [],
        "simulation_time": [],
        "computation_time": [],
    }

    df = pd.DataFrame(simulation_result_dataset)
    df.to_csv("cnn_simulation_result.csv")
    
def save_result(
    simulation_name,
    method_name,
    step_size,
    tolerance,
    total_iteration,
    simulation_time,
    computation_time
):
    df = pd.read_csv("cnn_simulation_result.csv", index_col=0)

    new_result = {
        "simulation_name": [simulation_name],
        "method_name": [method_name],
        "step_size": [step_size],
        "tolerance": [tolerance],
        "total_iteration": [total_iteration],
        "simulation_time": [simulation_time],
        "computation_time": [computation_time],
    }
    new_df = pd.DataFrame(new_result)
    df = pd.concat([df, new_df]).reset_index(drop=True)
    df.to_csv("cnn_simulation_result.csv")
    return new_result
generate_empty_simulation_result_csv()

In [15]:
simulation_name = "cnn_hole_fill_2"
ode_method = rkhem_system_method
ode_method_name = "rkhem_system_method"

steps = [.1, .25, .5, .75, 1, 1.25, 1.5, 1.75, 2, 2.25]
tols = [.5, .1, .05, .01, .005, .001, .0005, .0001, .00005]
hem_tols = [.5, .1, .05, .01, .005]
# hm_tols = [1.25, 1, .75, .5, .25, .1, .075, .05, .025, .01]
# tols = hm_tols
tols = hem_tols
# steps = [.75, 1]
# tols = [.01, .005]

grey_edge_detect_template = CNNTemplate(
    A=np.array(
        [[0, 0, 0],
        [0, 2, 0],
        [0, 0, 0]]
    ),
    B=np.array(
        [[-1, -1, -1],
        [-1, 8, -1],
        [-1, -1, -1]]
    ),
    I=-.5
)

for tol in tols:
    for step in steps:
        print(tol, step)
        cnn_input = ImageUtil.convert_image2bipolar_encode(Image.open("image/edge_detect_input_1.png"))
        cnn_state = np.ones(cnn_input.shape)
        cnn = CNN(cnn_input, cnn_state, grey_edge_detect_template)

        simulator = CNNSimulator(cnn)
        
        simulator.step_size = step
        simulator.simulation_time = 30
        simulator.max_tolerable_dynamic = float(.02)
        simulator.tol = float(tol)
        simulator.ode_method = ode_method
        
        simulator.simulate()
        
        iteration = simulator.get_num_of_iteration()
        simulation_time = simulator.simulation_time
        computation_time = simulator.computation_time
        result = save_result(
            simulation_name=simulation_name,
            method_name=ode_method_name,
            step_size=step,
            tolerance=tol,
            total_iteration=iteration,
            simulation_time=simulation_time,
            computation_time=computation_time,
        )

0.5 0.1
0.5 0.25
0.5 0.5
0.5 0.75
0.5 1
0.5 1.25
0.5 1.5
0.5 1.75
0.5 2
0.5 2.25
0.1 0.1
0.1 0.25
0.1 0.5
0.1 0.75
0.1 1
0.1 1.25
0.1 1.5
0.1 1.75
0.1 2
0.1 2.25
0.05 0.1
0.05 0.25
0.05 0.5
0.05 0.75
0.05 1
0.05 1.25
0.05 1.5
0.05 1.75
0.05 2
0.05 2.25
0.01 0.1
0.01 0.25
0.01 0.5
0.01 0.75
0.01 1
0.01 1.25
0.01 1.5
0.01 1.75
0.01 2
0.01 2.25
0.005 0.1
0.005 0.25
0.005 0.5
0.005 0.75
0.005 1
0.005 1.25
0.005 1.5
0.005 1.75
0.005 2
0.005 2.25
